https://github.com/dzakarya/machine_learning/blob/master/CNN/finetune.png

In [1]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import pathlib

In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)
print(data_dir)

/home/dewatic/.keras/datasets/flower_photos


In [3]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0, validation_split=0.4, dtype="float32")

In [4]:
b_size = 4
img_size = 100
trainset = datagen.flow_from_directory(data_dir,target_size=(img_size,img_size),color_mode='rgb',subset="training",batch_size=b_size,class_mode="categorical")
valset = datagen.flow_from_directory(data_dir,target_size=(img_size,img_size),color_mode='rgb',subset="validation",batch_size=b_size,class_mode="categorical")
print(trainset.class_indices)

Found 2204 images belonging to 5 classes.
Found 1466 images belonging to 5 classes.
{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}


In [5]:
vgg_model = tf.keras.applications.VGG16(weights="imagenet",include_top=False, input_shape = (img_size,img_size,3))

In [6]:
for layer in vgg_model.layers:
    layer.trainable = False
    print(layer)

In [7]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [19]:
x = vgg_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(100, activation="relu")(x)
x = tf.keras.layers.Dense(100, activation="relu")(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)
transfered_model = tf.keras.Model(inputs = vgg_model.input, outputs = x)
transfered_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0   

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
transfered_model.compile(optimizer=optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

In [22]:
transfered_model.fit(trainset,verbose=1,validation_data=valset,batch_size=32,epochs=10)

Epoch 1/10
551/551 [==============================] - 34s 61ms/step - loss: 0.1652 - accuracy: 0.9592 - val_loss: 5.5635 - val_accuracy: 0.7094
Epoch 2/10
551/551 [==============================] - 34s 62ms/step - loss: 0.1663 - accuracy: 0.9592 - val_loss: 5.6978 - val_accuracy: 0.7108
Epoch 3/10
551/551 [==============================] - 35s 63ms/step - loss: 0.1670 - accuracy: 0.9592 - val_loss: 6.0585 - val_accuracy: 0.7094
Epoch 4/10
551/551 [==============================] - 35s 63ms/step - loss: 0.1663 - accuracy: 0.9592 - val_loss: 6.6398 - val_accuracy: 0.7101
Epoch 5/10
551/551 [==============================] - 35s 63ms/step - loss: 0.2679 - accuracy: 0.9369 - val_loss: 5.5858 - val_accuracy: 0.6937
Epoch 6/10
551/551 [==============================] - 34s 62ms/step - loss: 0.3343 - accuracy: 0.9301 - val_loss: 6.4993 - val_accuracy: 0.7026
Epoch 7/10
551/551 [==============================] - 34s 62ms/step - loss: 0.2282 - accuracy: 0.9378 - val_loss: 5.4161 - val_accuracy: